### UC Berkeley, MICS, W202-Cryptography
### Week 03 Breakout 4

### Square Roots of a product of two prime numbers in modulo prime using brute force. Demonstrate that given 1 square root and p, q, we can recover the other 3 roots. Demonstrate that given n and 4 square roots, we can recover p and q, which is equivalent to factoring n into p and q.

In the last breakout, we saw how to find the square root in modulo prime.  We saw that square roots exist exactly 50% of the time for non-zero integers in modulo prime.  We also saw that square roots in modulo prime have a symmetrical pattern.  The lower half are expressed as positive numbers, while the upper half can be expressed as either a positive or negative number.  We also saw that each lower square root, if negated, matches an upper root.

In RSA, we used the n as the product of two prime numbers p and q, n = pq.  We saw that factoring has exponential computational complexity, and is computationally intractible for large values of n.  

In the next breakout, we will cover Rabin Signatures, which use square roots in modulo n, where n is the product of two prime numers, n = pq.  

In this breakout, we want to prepare ourselves, we will study:

* finding the square roots of a product of two prime numbers in modulo prime using brute force, using the same method as the previous breakout

* we will see that we only find roots approximately 15% of the time compared to the exact 50% in pure modulo prime

*  integers relatively prime to n will not have square roots modulo n

* we will see that if square roots exist, we always have exactly 4 square roots.  this should be intuitive, as we know p has 2 roots and q has 2 roots, so n = pq should have 4 roots

* if we know 1 of the 4 square roots, and we know p and q, we can recover the other 3 roots.  We take the one root and write it using a formula in the CRT notation.  We solve the CRT notation as 4 systems of simultaneous linear congruences, and this gives us all 4 roots, so we recover the other 3 roots.

* since p and q are kept secret, and we must know p and q to recover the other 3 roots, it should be intuitive as another example of how factoring is related to finding square roots.  If we can factor, we can find square roots.  If we can find square roots, we can factor.  So finding square roots is the same level of computational intractibility, and the same level of security, as factoring.  

* if we know n and 4 square roots, we can recover p and q, which is the equivalent of factoring n into p and q.

In [1]:
from sage.all import *

In [2]:
def my_find_squares_product_p_q(n):
    "given n = pq, with both p and q prime, list all of the squares (quadratic residues) in modulo n, and using CRT format"
    
    print ("\nn = " + str(n) + "\n")
    
    f = factor(n)
   
    if len(f) != 2:
        print ("n must be the product of 2 primes!")
        return
    
    if f[0][1] != 1 or f[1][1] != 1:
        print ("n must be the product of 2 primes!")
        return
    
    q = f[0][0]
    p = f[1][0]
    
    print ("p = " + str(p))
    print ("q = " + str(q))
    
    if not is_prime(p):
        print ("p must be prime!")
        return
    
    if not is_prime(q):
        print ("q must be prime!")
        return
    
    print ("\nSquares:\n")
    
    squares = []
    
    for i in range(1,n):
        if gcd(i,n) != 1:
            print (str(i) + " is not relatively prime to " + str(n))
        else:
            print (str(i) + "^2 = " + str((i ** 2) % n) + " (mod " + str(n) + ")")
            squares.append([i,(i ** 2) % n])
        
    quadratic_residues = []
        
    for s in squares:
        quadratic_residues.append(s[1])
        
    quadratic_residues = sorted(list(set(quadratic_residues)))
        
    print ("\nSquare Roots (all shown as positive numbers):\n")
    
    for i in range(1,n):
        
        if i not in quadratic_residues:
            
            if gcd(i,n) != 1:
                print (str(i) + " (mod " + str(n) + ") - NOT relatively prime to " + str(n))
            else:
                print (str(i) + " (mod " + str(n) + ") - square roots do NOT exist")
        
        else:
            
            s = str(i) + " (mod " + str(n) + ") has square roots:   "
            for sq in squares:
                if i == sq[1]:
                    s += str(sq[0]) + "   "
            print (s)
        
    print ("\nSquare Roots (secondary square root shown as both a positive and negative number):\n")
    
    for i in range(1,n):
        
        if i not in quadratic_residues:
            
            if gcd(i,n) != 1:
                print (str(i) + " (mod " + str(n) + ") - NOT relatively prime to " + str(n))
            else:
                print (str(i) + " (mod " + str(n) + ") - square roots do NOT exist")
        
        else:
            
            s = str(i) + " (mod " + str(n) + ") has square roots:   "
            count = 0
            for sq in squares:
                if i == sq[1]:
                    count += 1
                    if count <= 2:
                        s += str(sq[0]) + "   "
                    else:
                        s += str(sq[0]) + " = " + str(sq[0] - n) + " (mod " + str(n) + ")   "
            
            print (s)
        
    print ("\nNumber of squares: " + str(len(quadratic_residues)) + " of " + str(n-1) + " = " + \
                "{:.3g}".format(float(((len(quadratic_residues) / (n-1)) * 100.0))) + "%\n")
        
        
    print ("\nUsing the Chinese Remainder Theorem CRT Notation, if we can find 1 random root, we can recover the other 3.")
    print ("Below, for each square, we will take the first root that we found and show that we can recover the other 3.\n")
    
    for qr in quadratic_residues:
        
        print ("\n    Square Roots of " + str(qr) + " (mod " + str(n) + ")"+ " in CRT:\n")
        
        for sq in squares:
            
            if qr == sq[1]:
                r_root = sq[0]
                break
        
        print ("        random root r = ", r_root, "\n")
        
        print ("        r = < r mod p, r mod q> = < " + str(r_root) + " mod " + str(p) + ", " + str(r_root) + " mod " + str(q) + " >")
        print ("        s = < r mod p, -r mod q> = < " + str(r_root) + " mod " + str(p) + ", " + str(-r_root) + " mod " + str(q) + " >"  )
        print ("        -s = < -r mod p, r mod q> = < " + str(-r_root) + " mod " + str(p) + ", " + str(r_root) + " mod " + str(q) + " >")
        print ("        -r = < -r mod p, -r mod q> = < " + str(-r_root) + " mod " + str(p) + ", " + str(-r_root) + " mod " + str(q) + " >")

In [3]:
my_find_squares_product_p_q(15)


n = 15

p = 5
q = 3

Squares:

1^2 = 1 (mod 15)
2^2 = 4 (mod 15)
3 is not relatively prime to 15
4^2 = 1 (mod 15)
5 is not relatively prime to 15
6 is not relatively prime to 15
7^2 = 4 (mod 15)
8^2 = 4 (mod 15)
9 is not relatively prime to 15
10 is not relatively prime to 15
11^2 = 1 (mod 15)
12 is not relatively prime to 15
13^2 = 4 (mod 15)
14^2 = 1 (mod 15)

Square Roots (all shown as positive numbers):

1 (mod 15) has square roots:   1   4   11   14   
2 (mod 15) - square roots do NOT exist
3 (mod 15) - NOT relatively prime to 15
4 (mod 15) has square roots:   2   7   8   13   
5 (mod 15) - NOT relatively prime to 15
6 (mod 15) - NOT relatively prime to 15
7 (mod 15) - square roots do NOT exist
8 (mod 15) - square roots do NOT exist
9 (mod 15) - NOT relatively prime to 15
10 (mod 15) - NOT relatively prime to 15
11 (mod 15) - square roots do NOT exist
12 (mod 15) - NOT relatively prime to 15
13 (mod 15) - square roots do NOT exist
14 (mod 15) - square roots do NOT exist

Square R

In [4]:
my_find_squares_product_p_q(21)


n = 21

p = 7
q = 3

Squares:

1^2 = 1 (mod 21)
2^2 = 4 (mod 21)
3 is not relatively prime to 21
4^2 = 16 (mod 21)
5^2 = 4 (mod 21)
6 is not relatively prime to 21
7 is not relatively prime to 21
8^2 = 1 (mod 21)
9 is not relatively prime to 21
10^2 = 16 (mod 21)
11^2 = 16 (mod 21)
12 is not relatively prime to 21
13^2 = 1 (mod 21)
14 is not relatively prime to 21
15 is not relatively prime to 21
16^2 = 4 (mod 21)
17^2 = 16 (mod 21)
18 is not relatively prime to 21
19^2 = 4 (mod 21)
20^2 = 1 (mod 21)

Square Roots (all shown as positive numbers):

1 (mod 21) has square roots:   1   8   13   20   
2 (mod 21) - square roots do NOT exist
3 (mod 21) - NOT relatively prime to 21
4 (mod 21) has square roots:   2   5   16   19   
5 (mod 21) - square roots do NOT exist
6 (mod 21) - NOT relatively prime to 21
7 (mod 21) - NOT relatively prime to 21
8 (mod 21) - square roots do NOT exist
9 (mod 21) - NOT relatively prime to 21
10 (mod 21) - square roots do NOT exist
11 (mod 21) - square roots d

In [5]:
my_find_squares_product_p_q(33)


n = 33

p = 11
q = 3

Squares:

1^2 = 1 (mod 33)
2^2 = 4 (mod 33)
3 is not relatively prime to 33
4^2 = 16 (mod 33)
5^2 = 25 (mod 33)
6 is not relatively prime to 33
7^2 = 16 (mod 33)
8^2 = 31 (mod 33)
9 is not relatively prime to 33
10^2 = 1 (mod 33)
11 is not relatively prime to 33
12 is not relatively prime to 33
13^2 = 4 (mod 33)
14^2 = 31 (mod 33)
15 is not relatively prime to 33
16^2 = 25 (mod 33)
17^2 = 25 (mod 33)
18 is not relatively prime to 33
19^2 = 31 (mod 33)
20^2 = 4 (mod 33)
21 is not relatively prime to 33
22 is not relatively prime to 33
23^2 = 1 (mod 33)
24 is not relatively prime to 33
25^2 = 31 (mod 33)
26^2 = 16 (mod 33)
27 is not relatively prime to 33
28^2 = 25 (mod 33)
29^2 = 16 (mod 33)
30 is not relatively prime to 33
31^2 = 4 (mod 33)
32^2 = 1 (mod 33)

Square Roots (all shown as positive numbers):

1 (mod 33) has square roots:   1   10   23   32   
2 (mod 33) - square roots do NOT exist
3 (mod 33) - NOT relatively prime to 33
4 (mod 33) has square roots:  

In [6]:
my_find_squares_product_p_q(35)


n = 35

p = 7
q = 5

Squares:

1^2 = 1 (mod 35)
2^2 = 4 (mod 35)
3^2 = 9 (mod 35)
4^2 = 16 (mod 35)
5 is not relatively prime to 35
6^2 = 1 (mod 35)
7 is not relatively prime to 35
8^2 = 29 (mod 35)
9^2 = 11 (mod 35)
10 is not relatively prime to 35
11^2 = 16 (mod 35)
12^2 = 4 (mod 35)
13^2 = 29 (mod 35)
14 is not relatively prime to 35
15 is not relatively prime to 35
16^2 = 11 (mod 35)
17^2 = 9 (mod 35)
18^2 = 9 (mod 35)
19^2 = 11 (mod 35)
20 is not relatively prime to 35
21 is not relatively prime to 35
22^2 = 29 (mod 35)
23^2 = 4 (mod 35)
24^2 = 16 (mod 35)
25 is not relatively prime to 35
26^2 = 11 (mod 35)
27^2 = 29 (mod 35)
28 is not relatively prime to 35
29^2 = 1 (mod 35)
30 is not relatively prime to 35
31^2 = 16 (mod 35)
32^2 = 9 (mod 35)
33^2 = 4 (mod 35)
34^2 = 1 (mod 35)

Square Roots (all shown as positive numbers):

1 (mod 35) has square roots:   1   6   29   34   
2 (mod 35) - square roots do NOT exist
3 (mod 35) - square roots do NOT exist
4 (mod 35) has square roots

#### Let's take an example of square roots written in CRT notation, and solve the system of linear congruences to show that we can indeed recover all 4 roots if we only know 1 random root and we know p and q


Square Roots of 11 (mod 35) in CRT:

random root r =  9 

r = < r mod p, r mod q> = < 9 mod 7, 9 mod 5 >
        
s = < r mod p, -r mod q> = < 9 mod 7, -9 mod 5 >
        
-s = < -r mod p, r mod q> = < -9 mod 7, 9 mod 5 >
        
-r = < -r mod p, -r mod q> = < -9 mod 7, -9 mod 5 >

Square roots previous found above:

11 (mod 35) has square roots:   9   16   19 = -16 (mod 35)   26 = -9 (mod 35)   

Note: some of the congruences may not be in simpliest form, so we will need to cut by a mod before solving the systems of linear congruences.

In [7]:
def my_solve_crt(congruence_list):
    "Given a list containing a system of linear congruences (residue, modulus) solve the system using brute force"
    
    print ("\nCongruences:")
    for (residue, modulus) in congruence_list:
        print ("    x congruent " + str(residue) + " (mod " + str(modulus) + ")")
    
    modulus_product = 1
    
    start_value = 0
    
    for (residue, modulus) in congruence_list:
        modulus_product *= modulus
        if residue > start_value:
            start_value = residue
        
    
    print ("\nstart value:", start_value)
    print ("stop value :", modulus_product, "\n(inclusive)\n")
    
    if modulus_product > 200:
        print ("won't print the list since modulus product is > 200\n")
    
    found = False
    solution = -1
    
    for i in range(start_value, modulus_product):
        for (residue, modulus) in congruence_list:
            if (i % modulus) == residue:
                found = True
                solution = i
            else:
                found = False
                if modulus_product < 200:
                    print ("    ", i, "is NOT a solution")
                break
        if found:
            solution = i
            break
            
    if found:
        print ("\n***", solution, "is the solution ***")
        for (residue, modulus) in congruence_list:
            print ("    " + str(solution) + " (mod " + str(modulus) + ") = " + str(residue))
    else:
        print ("\n*** NO SOLUTION ***")

In [8]:
# 11 (mod 35) has square roots:   9   16   19 = -16 (mod 35)   26 = -9 (mod 35)   

# r = < r mod p, r mod q> = < 9 mod 7, 9 mod 5 >

my_solve_crt([(9%7,7),(9%5,5)])


Congruences:
    x congruent 2 (mod 7)
    x congruent 4 (mod 5)

start value: 4
stop value : 35 
(inclusive)

     4 is NOT a solution
     5 is NOT a solution
     6 is NOT a solution
     7 is NOT a solution
     8 is NOT a solution

*** 9 is the solution ***
    9 (mod 7) = 2
    9 (mod 5) = 4


In [9]:
# 11 (mod 35) has square roots:   9   16   19 = -16 (mod 35)   26 = -9 (mod 35)   

# s = < r mod p, -r mod q> = < 9 mod 7, -9 mod 5 >

my_solve_crt([(9%7,7),(-9%5,5)])


Congruences:
    x congruent 2 (mod 7)
    x congruent 1 (mod 5)

start value: 2
stop value : 35 
(inclusive)

     2 is NOT a solution
     3 is NOT a solution
     4 is NOT a solution
     5 is NOT a solution
     6 is NOT a solution
     7 is NOT a solution
     8 is NOT a solution
     9 is NOT a solution
     10 is NOT a solution
     11 is NOT a solution
     12 is NOT a solution
     13 is NOT a solution
     14 is NOT a solution
     15 is NOT a solution

*** 16 is the solution ***
    16 (mod 7) = 2
    16 (mod 5) = 1


In [10]:
# 11 (mod 35) has square roots:   9   16   19 = -16 (mod 35)   26 = -9 (mod 35)   

# -s = < -r mod p, r mod q> = < -9 mod 7, 9 mod 5 >

my_solve_crt([(-9%7,7),(9%5,5)])



Congruences:
    x congruent 5 (mod 7)
    x congruent 4 (mod 5)

start value: 5
stop value : 35 
(inclusive)

     5 is NOT a solution
     6 is NOT a solution
     7 is NOT a solution
     8 is NOT a solution
     9 is NOT a solution
     10 is NOT a solution
     11 is NOT a solution
     12 is NOT a solution
     13 is NOT a solution
     14 is NOT a solution
     15 is NOT a solution
     16 is NOT a solution
     17 is NOT a solution
     18 is NOT a solution

*** 19 is the solution ***
    19 (mod 7) = 5
    19 (mod 5) = 4


In [11]:
# 11 (mod 35) has square roots:   9   16   19 = -16 (mod 35)   26 = -9 (mod 35)   

# -r = < -r mod p, -r mod q> = < -9 mod 7, -9 mod 5 >

my_solve_crt([(-9%7,7),(-9%5,5)])



Congruences:
    x congruent 5 (mod 7)
    x congruent 1 (mod 5)

start value: 5
stop value : 35 
(inclusive)

     5 is NOT a solution
     6 is NOT a solution
     7 is NOT a solution
     8 is NOT a solution
     9 is NOT a solution
     10 is NOT a solution
     11 is NOT a solution
     12 is NOT a solution
     13 is NOT a solution
     14 is NOT a solution
     15 is NOT a solution
     16 is NOT a solution
     17 is NOT a solution
     18 is NOT a solution
     19 is NOT a solution
     20 is NOT a solution
     21 is NOT a solution
     22 is NOT a solution
     23 is NOT a solution
     24 is NOT a solution
     25 is NOT a solution

*** 26 is the solution ***
    26 (mod 7) = 5
    26 (mod 5) = 1


### Let's take an example where we know n and we know 4 square roots.  (We do not know p and q in this example)  We can recover p and q, which is the same as factoring n into p and q.

* Given: n and 4 square roots of a number in modulo n

* make a list of all unique pairs of square roots (there will be 6 of them)

* for each pair s and t, use the Euclidean Algorithm to find gcd(s + t, n)

* use the positive values for s and t

* if it yields a number < n, it will be a factor, either p or q

* if it yields n, we learn no informaiton about p nor q

Let's use the example of 11 (mod 35):

11 (mod 35) has square roots:   9   16   19 = -16 (mod 35)   26 = -9 (mod 35) 

In [12]:
def my_given_n_4_roots_recover_p_q(n, square_roots):
    "given n the product of two primes p and q, and 4 square roots of a number, recover p and q"
    
    print ("\nn = " + str(n))
    print ("square roots = " + str(square_roots))
    
    i = 0
    
    while i < 4:
        
        j = i + 1
        
        while j < 4:
            
            s = square_roots[i]
            t = square_roots[j]
            my_gcd = gcd(s+t,n)
            
            print ("\n")
            print ("s = " + str(s))
            print ("t = " + str(t))
            print ("gcd(s+t,n) = gcd(" + str(s) + " + " + str(t) + ", " + str(n) + ") = " + str(my_gcd))
            
            if my_gcd == n:
                print ("we did NOT recover a factor")
            else:
                print ("we recovered a factor:", my_gcd)
                print ("we can recover the other factor: n / " + str(my_gcd) + " = " + str(n) + " / " + str(my_gcd) + " = " + str(n // my_gcd))
            
            j += 1
            
        i += 1


In [13]:
# 11 (mod 35) has square roots: 9 16 19 = -16 (mod 35) 26 = -9 (mod 35)

my_given_n_4_roots_recover_p_q(35, [9, 16, 19, 26])


n = 35
square roots = [9, 16, 19, 26]


s = 9
t = 16
gcd(s+t,n) = gcd(9 + 16, 35) = 5
we recovered a factor: 5
we can recover the other factor: n / 5 = 35 / 5 = 7


s = 9
t = 19
gcd(s+t,n) = gcd(9 + 19, 35) = 7
we recovered a factor: 7
we can recover the other factor: n / 7 = 35 / 7 = 5


s = 9
t = 26
gcd(s+t,n) = gcd(9 + 26, 35) = 35
we did NOT recover a factor


s = 16
t = 19
gcd(s+t,n) = gcd(16 + 19, 35) = 35
we did NOT recover a factor


s = 16
t = 26
gcd(s+t,n) = gcd(16 + 26, 35) = 7
we recovered a factor: 7
we can recover the other factor: n / 7 = 35 / 7 = 5


s = 19
t = 26
gcd(s+t,n) = gcd(19 + 26, 35) = 5
we recovered a factor: 5
we can recover the other factor: n / 5 = 35 / 5 = 7


### Summary: 

* if we can factor n into p and q, we can find square roots in modulo n.  

* If we can find square roots in modulo n, we can factor n into p and q.  

* Factoring n into p and q is the same as find square roots in modulo n